In [1]:
# --- Imports ---
import pymif.microscope_manager as mm
import napari
import os
import time

In [2]:
# --- Set input data path ---
main_path = "../"
dataset_name = "E_for_Nicola"

outpath = "../"
outdataset_name = "cephalopods_ome-zarr_V05_with_groups_and_labels"

# main_path = "/g/mif/common/Data/Z_from_LCS_24-07-16/Coll_Montse/MouseEmbryo_micemetyleneBlue/2022-08-09_132304/processed"
# dataset_name = "20221108-162726_Task_2_LM_FullRes_C"

# main_path = "/mif-users/Users/Arias_Adrian/scans_CSL_P21/2400_31573_s"
# dataset_name = "20250523-105843_Task_1_stitching_1_C"

data_path = f"{main_path}/{dataset_name}"

In [3]:
output_zarr_path = f"{outpath}/{outdataset_name}.zarr"
zarrdataset = mm.ZarrManager(output_zarr_path, mode="a")

/
├── 0 (1, 3, 512, 672, 512) uint16
├── labels
│   └── nuclei
│       └── 0 (1, 512, 672, 512) uint16
└── test
    └── 0 (1, 1, 512, 672, 512) uint16


SIZE: [(1, 3, 512, 672, 512)]
CHUNKSIZE: [(1, 1, 512, 672, 512)]
SCALES: [(15.33, 15.33, 15.33)]
UNITS: ('micrometer', 'micrometer', 'micrometer')
TIME_INCREMENT: 1.0
TIME_INCREMENT_UNIT: s
CHANNEL_NAMES: ['Trans', 'GFP', 'Cy5']
CHANNEL_COLORS: [16777215, 65280, 16711935]
DTYPE: uint16
PLANE_FILES: None
AXES: tczyx


# Use create_empty_group to create a processed group

In [ ]:
metadata = {
    "axes" : "tczyx",
    "size" : [tuple([s[0],1,s[2],s[3],s[4]]) for s in zarrdataset.metadata["size"]],
    "chunksize" : zarrdataset.metadata["chunksize"],
    "scales" : zarrdataset.metadata["scales"],
    "time_increment" : zarrdataset.metadata["time_increment"],
    "time_increment_unit" : zarrdataset.metadata["time_increment_unit"],
    "units" : zarrdataset.metadata["units"],
    "channel_names" : ["GFP_processed"],
    "channel_colors" : [16711935],
}


In [ ]:
zarrdataset.create_empty_group("test", metadata=metadata)

In [ ]:
from skimage.filters import gaussian
array = zarrdataset.data[0][0,1,...].compute()
array = gaussian(array, sigma=1, preserve_range=True)
array = array[None,None,...]
print(array.shape)

In [ ]:
zarrdataset.write_image_region(array,
                              t=slice(0,1), c=slice(0,1), z=slice(0,zarrdataset.data[0].shape[2]),
                              y=slice(0,zarrdataset.data[0].shape[3]),
                              x=slice(0,zarrdataset.data[0].shape[4]),
                              group="test")
                               

In [ ]:
z = mm.ZarrManager("../cephalopods_ome-zarr_V05_with_grups_and_labels.zarr", mode="a")

In [ ]:
print(z.data)
print(z.groups)
print(z.labels)

# Use create_empty_group to create label

In [ ]:
metadata = {
    "axes" : "tczyx",
    "size" : [tuple([s[0],s[2],s[3],s[4]]) for s in zarrdataset.metadata["size"]],
    "chunksize" : [tuple([s[0],s[2],s[3],s[4]]) for s in zarrdataset.metadata["chunksize"]],
    "scales" : zarrdataset.metadata["scales"],
    "time_increment" : zarrdataset.metadata["time_increment"],
    "time_increment_unit" : zarrdataset.metadata["time_increment_unit"],
    "units" : zarrdataset.metadata["units"],
    "channel_names" : ["GFP_processed"],
    "channel_colors" : [16711935],
}

In [ ]:
zarrdataset.create_empty_group("nuclei", metadata=metadata, is_label=True)

In [ ]:
import numpy as np
array = zarrdataset.data[0][0,1,...].compute()
array = array>np.mean(array)
array = array[None,...]
print(array.shape)

In [ ]:
zarrdataset.write_label_region(array,
                              t=slice(0,1), z=slice(0,zarrdataset.data[0].shape[2]),
                              y=slice(0,zarrdataset.data[0].shape[3]),
                              x=slice(0,zarrdataset.data[0].shape[4]),
                              group="labels/nuclei")

# Test that loading works well

In [ ]:
output_zarr_path = f"{outpath}/{outdataset_name}.zarr"
zarrdataset = mm.ZarrManager(output_zarr_path, mode="a")

In [ ]:
print(zarrdataset.data)
print(zarrdataset.groups)
print(zarrdataset.labels)
print(zarrdataset.metadata)

# Check create_empty_dataset

In [4]:
output_zarr_path = f"{outpath}/{outdataset_name}_new.zarr"
zarr_dataset_copy = mm.ZarrManager(path=output_zarr_path, mode="w", metadata=zarrdataset.metadata)

[INFO] Created empty OME-Zarr dataset at file://../cephalopods_ome-zarr_V05_with_groups_and_labels_new.zarr.


### Fill data

In [5]:
vol = zarrdataset.data[0]

In [6]:
vol

dask.array<from-zarr, shape=(1, 3, 512, 672, 512), dtype=uint16, chunksize=(1, 1, 512, 672, 512), chunktype=numpy.ndarray>

In [7]:
zarr_dataset_copy.write_image_region(vol,
                              t=slice(0,1), c=slice(0,3), z=slice(0,vol.shape[2]),
                              y=slice(0,vol.shape[3]),
                              x=slice(0,vol.shape[4]),group="/")

# Check add_labels

In [8]:
lbl = zarrdataset.labels["nuclei"]

In [9]:
lbl[0]

dask.array<from-zarr, shape=(1, 512, 672, 512), dtype=uint16, chunksize=(1, 512, 672, 512), chunktype=numpy.ndarray>

In [10]:
zarr_dataset_copy.add_label(lbl, "nuclei")